In [2]:
from data_loader import get_task1_conver, get_task2_conver, preprocess

In [3]:
import pandas as pd

In [4]:
import sys
sys.path.append('..')

from utils import load_jsonl, dump_jsonl, set_random_seed

In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import wandb
# from sklearn.metrics import precision_recall_fscore_support, classification_report
# from pythainlp.tokenize import word_tokenize
import torch
import datasets
from datasets import Dataset, DatasetDict
from transformers import DataCollatorWithPadding
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer



In [6]:
from sklearn.utils import compute_class_weight
import torch.nn as nn
from itertools import groupby
import os, shutil
    
def preprocess(text):
    text = text.lower()

    # handle repeted charecters
    s = ""
    for k,v in groupby(text):
        n = len(list(v))
        s += k*min(n, 4)
        if n >= 4:
          s += " rep "

    text = s
    
    return text

In [7]:
from sklearn.metrics import f1_score, precision_recall_fscore_support, confusion_matrix, accuracy_score
from tqdm import tqdm

def run_eval(best_ckpt_path, df):

    # set_random_seed()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # device = torch.device("cpu")
    print("START")
    print("step 1: load data")

    # print(train.groupby("label").count())
    # print(test.groupby("label").count())

    print("step 2: load tokenizer")
    model_name = "airesearch/wangchanberta-base-att-spm-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    num_added_toks = tokenizer.add_special_tokens({"additional_special_tokens": ["usr", "sys", "rep"]})

    print("step 3: init data")
    ds = DatasetDict()
    ds['test'] = Dataset.from_pandas(df)

    print("step 4: load model")
    model = AutoModelForSequenceClassification.from_pretrained(best_ckpt_path);
    model = model.to(device)

    id2label = model.config.id2label
    label2id = model.config.label2id

#     def word_tokenize(d, tokenizer=None, label2id=None, max_length=256):
#         tokens = tokenizer(d["text"], truncation=True, max_length=max_length)
#         tokens["label"] = [label2id[label] for label in d["label"]]
#         return tokens

#     tokenized_ds = ds.map(word_tokenize, batched=True, fn_kwargs={"tokenizer":tokenizer, "label2id": label2id, "max_length":max_length})
#     data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    print("step 6: evaluate")


    y_true, y_pred = [], []
    for t, l in tqdm(zip(df["text"], df["label"]), total=len(df["text"])):
        inputs = tokenizer(preprocess(t), return_tensors="pt", truncation=True, padding="max_length", max_length=max_length).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits

        predicted_class_id = logits.argmax().item()
        p = model.config.id2label[predicted_class_id]


        y_true.append(l)
        y_pred.append(p)
    

    print("F1", f1_score(y_true, y_pred, average="macro"))
    print("P/R/F1", precision_recall_fscore_support(y_true, y_pred, average="macro"))
    print("Acc", accuracy_score(y_true, y_pred))
    return y_true, y_pred

In [8]:
report = "none"
batch_size = 16
max_length = 128
num_epochs = 5

In [9]:
# Why are precision, recall and F1 score equal when using micro averaging in a multi-class problem?
# https://simonhessner.de/why-are-precision-recall-and-f1-score-equal-when-using-micro-averaging-in-a-multi-class-problem/

## Task1

In [98]:
train, val, test = get_task1_conver("../Task1/annotated_conersations.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
model_path = "./Models/task1_clse_usr"
for name, df in [("train", train), ("val", val), ("test", test)]:
    print("Dataset", name)
    y_true, y_pred = run_eval(f"{model_path}/best_model", df);
    df["predict"] = y_pred
    df.to_csv(f"{model_path}/{name}.csv", index=False)
    print("======================")

Loaded 1234 records from ../Task1/annotated_conersations.jsonl
N 1096 60 60
Dataset train
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1096/1096 [00:16<00:00, 67.92it/s]


F1 0.7895186959860322
P/R/F1 (0.7980044241249482, 0.7871498187256325, 0.7895186959860322, None)
Acc 0.8366788321167883
Dataset val
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 68.06it/s]


F1 0.5541666666666667
P/R/F1 (0.5404761904761904, 0.6129629629629629, 0.5541666666666667, None)
Acc 0.7333333333333333
Dataset test
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 68.00it/s]

F1 0.3947746195808211
P/R/F1 (0.4102564102564103, 0.3871158392434988, 0.3947746195808211, None)
Acc 0.6666666666666666


In [99]:
train, val, test = get_task1_conver("../Task1/annotated_conersations.jsonl", "authority", skips = ["3. Not respect"], only_user=True)
model_path = "./Models/task1_auth_usr"
for name, df in [("train", train), ("val", val), ("test", test)]:
    print("Dataset", name)
    y_true, y_pred = run_eval(f"{model_path}/best_model", df);
    df["predict"] = y_pred
    df.to_csv(f"{model_path}/{name}.csv", index=False)
    print("======================")

Loaded 1234 records from ../Task1/annotated_conersations.jsonl
N 1098 61 61
Dataset train
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1098/1098 [00:16<00:00, 68.08it/s]


F1 0.5431583686635778
P/R/F1 (0.5430539553061182, 0.5717105834350583, 0.5431583686635778, None)
Acc 0.5737704918032787
Dataset val
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 68.10it/s]


F1 0.4399484374017481
P/R/F1 (0.44868644868644864, 0.44206349206349205, 0.4399484374017481, None)
Acc 0.5573770491803278
Dataset test
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 68.04it/s]

F1 0.36638245041606393
P/R/F1 (0.37103174603174605, 0.3637037037037037, 0.36638245041606393, None)
Acc 0.3770491803278688


## Task2

In [100]:
train, val, test = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
model_path = "./Models/task2_clse_usr"
for name, df in [("train", train), ("val", val), ("test", test)]:
    print("Dataset", name)
    y_true, y_pred = run_eval(f"{model_path}/best_model", df);
    df["predict"] = y_pred
    df.to_csv(f"{model_path}/{name}.csv", index=False)
    print("======================")

Loaded 2463 records from ../Task2/annotated/annotated.jsonl
N 1495 186 186
Dataset train
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1495/1495 [00:21<00:00, 69.98it/s]


F1 0.6320846447983318
P/R/F1 (0.5863550450415173, 0.7598472960767585, 0.6320846447983318, None)
Acc 0.7598662207357859
Dataset val
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 186/186 [00:02<00:00, 69.89it/s]


F1 0.4827057087731245
P/R/F1 (0.49442860288155344, 0.515220700152207, 0.4827057087731245, None)
Acc 0.6559139784946236
Dataset test
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 186/186 [00:02<00:00, 70.05it/s]

F1 0.4694461789009754
P/R/F1 (0.4980901451489687, 0.4710466838126413, 0.4694461789009754, None)
Acc 0.6182795698924731


In [101]:
train, val, test = get_task2_conver("../Task2/annotated/annotated.jsonl", "authority", skips = [], only_user=True)
model_path = "./Models/task2_auth_usr"
for name, df in [("train", train), ("val", val), ("test", test)]:
    print("Dataset", name)
    y_true, y_pred = run_eval(f"{model_path}/best_model", df);
    df["predict"] = y_pred
    df.to_csv(f"{model_path}/{name}.csv", index=False)
    print("======================")

Loaded 2463 records from ../Task2/annotated/annotated.jsonl
N 1642 205 205
Dataset train
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1642/1642 [00:23<00:00, 70.03it/s]


F1 0.8535997625931794
P/R/F1 (0.8125082341261645, 0.915890224106319, 0.8535997625931794, None)
Acc 0.8861144945188795
Dataset val
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 205/205 [00:02<00:00, 69.87it/s]


F1 0.7958696340338852
P/R/F1 (0.7944205531840615, 0.8068606825619448, 0.7958696340338852, None)
Acc 0.8390243902439024
Dataset test
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 205/205 [00:02<00:00, 69.87it/s]

F1 0.7648739164696612
P/R/F1 (0.7783709434773264, 0.7665112665112664, 0.7648739164696612, None)
Acc 0.7707317073170732


## Tasl3

In [10]:
train, val, test = get_task1_conver("../Task3/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
model_path = "./Models/task3_clse_usr"
for name, df in [("train", train), ("val", val), ("test", test)]:
    print("Dataset", name)
    y_true, y_pred = run_eval(f"{model_path}/best_model", df);
    df["predict"] = y_pred
    df.to_csv(f"{model_path}/{name}.csv", index=False)
    print("======================")

Loaded 1221 records from ../Task3/annotated/annotated.jsonl
N 1090 60 60
Dataset train
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1090/1090 [00:16<00:00, 67.79it/s]


F1 0.7831724677045807
P/R/F1 (0.7548506780118812, 0.8331039521108013, 0.7831724677045807, None)
Acc 0.865137614678899
Dataset val
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 68.12it/s]


F1 0.674883674883675
P/R/F1 (0.7015015015015015, 0.6564722617354196, 0.674883674883675, None)
Acc 0.75
Dataset test
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 68.13it/s]
/home/imtk/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1 0.38003565062388595
P/R/F1 (0.39829059829059826, 0.375, 0.38003565062388595, None)
Acc 0.65


In [12]:
train, val, test = get_task1_conver("../Task3/annotated/annotated.jsonl", "authority", skips = [], only_user=True)
model_path = "./Models/task3_auth_usr"
for name, df in [("train", train), ("val", val), ("test", test)]:
    print("Dataset", name)
    y_true, y_pred = run_eval(f"{model_path}/best_model", df);
    df["predict"] = y_pred
    df.to_csv(f"{model_path}/{name}.csv", index=False)
    print("======================")

Loaded 1221 records from ../Task3/annotated/annotated.jsonl
N 1099 61 61
Dataset train
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1099/1099 [00:16<00:00, 67.89it/s]


F1 0.750098843174169
P/R/F1 (0.6976094706215709, 0.8554852554852554, 0.750098843174169, None)
Acc 0.7989080982711556
Dataset val
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 67.90it/s]


F1 0.7028909787530476
P/R/F1 (0.6583333333333333, 0.7932624113475177, 0.7028909787530476, None)
Acc 0.819672131147541
Dataset test
START
step 1: load data
step 2: load tokenizer
step 3: init data
step 4: load model
step 6: evaluate


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 68.01it/s]


F1 0.813040293040293
P/R/F1 (0.795791487326638, 0.8916666666666666, 0.813040293040293, None)
Acc 0.819672131147541
